In [1]:
from pathlib import Path
import pandas as pd

# pip install git+https://github.com/boudinfl/pke.git
# pke relies on spacy (>= 3.2.3) for text processing and requires models to be installed:
# # download the russian model
# python -m spacy download ru_core_news_lg
import pke
import string
import ru_core_news_lg
import spacy

In [ ]:
!python -m spacy download ru_core_news_lg

In [2]:
spacy_model = spacy.load("ru_core_news_lg")

In [3]:
all_df = pd.read_csv("data/all_dataset.csv", sep="\t", encoding="utf-8")

In [4]:
all_df = all_df[all_df["rubrics"].str.contains('образование_отзывус')].dropna().astype("str").reset_index()

In [37]:
text = []
for i in range(len(all_df)):
    text.append([
        (w.text, w.pos_)
        for w in spacy_model(all_df['text'][i])
    ])

text_normalized = []
for i in range(len(all_df)):
    text_normalized.append([
        (w.lemma_, w.pos_)
        for w in spacy_model(all_df['text'][i])
        if w.pos_ != "PUNCT"
    ])

In [41]:
print(text[0], '\n')
print(text_normalized[0])

[('Был', 'AUX'), ('скучноват', 'ADJ'), ('.', 'PUNCT'), ('Много', 'ADV'), ('практики', 'NOUN'), (',', 'PUNCT'), ('решение', 'NOUN'), ('задач', 'NOUN'), ('про', 'ADP'), ('заработную', 'ADJ'), ('плату', 'NOUN'), ('.', 'PUNCT'), ('Довольно', 'ADV'), ('сложно', 'ADJ'), ('зарабатывать', 'VERB'), ('баллы', 'NOUN'), (',', 'PUNCT'), ('нужно', 'ADJ'), ('теорию', 'NOUN'), ('учить', 'VERB'), ('и', 'CCONJ'), ('местами', 'NOUN'), ('её', 'PRON'), ('рассказывать', 'VERB'), (',', 'PUNCT'), ('решать', 'VERB'), ('постоянно', 'ADV'), ('задачи', 'NOUN'), (',', 'PUNCT'), ('выходить', 'VERB'), ('к', 'ADP'), ('доске', 'NOUN'), ('.', 'PUNCT'), ('Когда', 'SCONJ'), ('баллы', 'NOUN'), ('набраны', 'VERB'), ('на', 'ADP'), ('зачёт', 'NOUN'), (',', 'PUNCT'), ('все', 'ADV'), ('равно', 'ADJ'), ('нужно', 'ADJ'), ('ходить', 'VERB'), (',', 'PUNCT'), ('иначе', 'CCONJ'), ('она', 'PRON'), ('их', 'PRON'), ('уберёт', 'VERB'), ('(', 'PUNCT'), ('хотя', 'SCONJ'), (',', 'PUNCT'), ('она', 'PRON'), ('просто', 'PART'), ('так', 'ADV')

# TextRank

In [46]:
%%time

# define the set of valid Part-of-Speeches
# pos = {'NOUN', 'PROPN', 'ADJ'}
pos = {'NOUN', 'PROPN'}

# 1. create a TextRank extractor.
extractor_textrank = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor_textrank.load_document(input=text,
                        language='ru',
                        #spacy_model=spacy_model,
                        normalization=None)

extractor_textrank.ngram_selection(n=1)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor_textrank.candidate_weighting(window=3,
                              pos=pos,
                              top_percent=0.33)


CPU times: user 2.53 s, sys: 4.01 ms, total: 2.53 s
Wall time: 2.53 s


In [47]:
# 4. get the 10-highest scored candidates as keyphrases
keyphrases_textrank = extractor_textrank.get_n_best(n=100)
keyphrases_textrank

[('онлайн - прокторинг -', 0.03583410910791599),
 ('- студентов электив', 0.030753442973738512),
 ('теории - электив', 0.029691802561052253),
 ('онлайн - электив', 0.02948812307463321),
 ('электив - это дело', 0.028988884305596915),
 ('электив - просто', 0.02879535296108404),
 ('- электив', 0.028288105446606277),
 ('электив -', 0.028123595446606277),
 ('преподаватель - практик', 0.0279775104922519),
 ('- преподаватель', 0.02774426192581579),
 ('преподаватель - очень', 0.027574762066173538),
 ('проблема - преподаватель', 0.027241251511455743),
 ('преподаватель -', 0.02699063192581579),
 ('все задания - зачёт', 0.023427296031799674),
 ('конце курса тест - прокторинг', 0.02333913578941216),
 ('задания - зачет', 0.02218465178065258),
 ('вопросов - доп баллы', 0.021676386350829423),
 ('- задания', 0.021472031523587416),
 ('баллы - иди', 0.021265226859483073),
 ('минус - баллы', 0.021189581766089885),
 ('конце семестра -', 0.021139268789660517),
 ('- материала лекций', 0.02104206334880842),


# TopicRank

In [48]:
%%time

# initialize keyphrase extraction model, here TopicRank
extractor_topicrank = pke.unsupervised.TopicRank()

# 2. load the content of the document.
stoplist = list(string.punctuation)
stoplist += pke.lang.stopwords.get('ru')

# load the content of the document, here document is expected to be a simple
# test string and preprocessing is carried out using spacy
extractor_topicrank.load_document(input=text, language='ru', stoplist=stoplist)

# keyphrase candidate selection, in the case of TopicRank: sequences of nouns
# and adjectives (i.e. `(Noun|Adj)*`)
pos = {'NOUN', 'PROPN'}
extractor_topicrank.candidate_selection(pos=pos)

# candidate weighting, in the case of TopicRank: using a random walk algorithm
extractor_topicrank.candidate_weighting(threshold=0.74, method='average')


CPU times: user 3min 59s, sys: 400 ms, total: 4min
Wall time: 4min


In [49]:
# N-best selection, keyphrases contains the 10 highest scored candidates as
# (keyphrase, score) tuples
keyphrases_topicrank = extractor_topicrank.get_n_best(n=100)
keyphrases_topicrank

[('электив', 0.04409892532663443),
 ('преподаватель', 0.029534003825206948),
 ('пары', 0.0272185985541718),
 ('задания', 0.019516724061802812),
 ('лекции', 0.014739990517597938),
 ('баллы', 0.013186711651915472),
 ('работы', 0.012227015477453375),
 ('элективом', 0.011806833942564349),
 ('тему постправды', 0.011566622566635822),
 ('занятия', 0.010640114140423165),
 ('практики', 0.009906862723116689),
 ('курса', 0.009373925279679672),
 ('студентам', 0.00895001895559797),
 ('презентации', 0.008662835242546272),
 ('информации', 0.008462557183091593),
 ('зачёт', 0.008217251356580718),
 ('тест', 0.007824342205323331),
 ('вопросы', 0.007529979439361245),
 ('материал', 0.007488542227163388),
 ('уровень знаний', 0.007200138076143224),
 ('конце', 0.006490403468946084),
 ('целом', 0.0062688927750305335),
 ('предмета', 0.0057758029570162345),
 ('языке', 0.005055112289460084),
 ('проект', 0.004965537596413936),
 ('группы', 0.004845708532106312),
 ('фильмы', 0.0046390186099258485),
 ('время', 0.0045

# SingleRank

In [50]:
%%time

# 1. create a SingleRank extractor.
extractor_singlerank = pke.unsupervised.SingleRank()

# 2. load the content of the document.
extractor_singlerank.load_document(input=text,
                        language='ru',
                        normalization=None)

# 3. select the longest sequences of nouns and adjectives as candidates.
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN'}
extractor_singlerank.candidate_selection(pos=pos)

# 4. weight the candidates using the sum of their word's scores that are
#    computed using random walk. In the graph, nodes are words of
#    certain part-of-speech (nouns and adjectives) that are connected if
#    they occur in a window of 10 words.
extractor_singlerank.candidate_weighting(window=3,
                                         pos=pos)


CPU times: user 635 ms, sys: 7.98 ms, total: 643 ms
Wall time: 652 ms


In [51]:
# 5. get the 100-highest scored candidates as keyphrases
keyphrases_singlerank = extractor_singlerank.get_n_best(n=100)
keyphrases_singlerank

[('преподавательница - елена эриковна -', 0.046004901786190926),
 ('онлайн - прокторинг - сдача экзамена', 0.04406617339124258),
 ('бла - бла - бла', 0.04308016674203402),
 ('неайти - студентов электив', 0.0370307300246557),
 ('онлайн - электив', 0.035825442729248674),
 ('электив - сплошь социология', 0.035063200618192965),
 ('лесу - электив', 0.03504466412641493),
 ('электив - чудо', 0.03504043336409616),
 ('- электив', 0.03485377302565539),
 ('электив -', 0.03440385302565539),
 ('электив - пытка', 0.034336644126414934),
 ('преподаватель наталья георгиевна -', 0.03406444956773917),
 ('преподаватель - практик', 0.03362483999349988),
 ('- преподаватель', 0.03342529252852009),
 ('преподаватель - психолог', 0.03340576070698332),
 ('преподаватель - замечательная', 0.033244568832772485),
 ('преподаватель петров в.в. -', 0.033198166364984946),
 ('преподаватель - грамотный специалист', 0.033016647603976684),
 ('преподаватель - крутая', 0.03295540883277249),
 ('проблема - преподаватель', 0.032

# PositionRank

In [52]:
%%time

# define the valid Part-of-Speeches to occur in the graph
pos = {'NOUN', 'PROPN'}

# define the grammar for selecting the keyphrase candidates
grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"
# grammar = "NP: {<NOUN|PROPN>+}"

# 1. create a PositionRank extractor.
extractor_positionrank = pke.unsupervised.PositionRank()

# 2. load the content of the document.
extractor_positionrank.load_document(input=text,
                                     language='ru',
                                     normalization=None)

# 3. select the noun phrases up to 3 words as keyphrase candidates.
extractor_positionrank.candidate_selection(grammar=grammar,
                                           maximum_word_number=3)

# 4. weight the candidates using the sum of their word's scores that are
#    computed using random walk biaised with the position of the words
#    in the document. In the graph, nodes are words (nouns and
#    adjectives only) that are connected if they occur in a window of
#    10 words.
extractor_positionrank.candidate_weighting(window=3,
                                           pos=pos)


CPU times: user 922 ms, sys: 20 ms, total: 942 ms
Wall time: 940 ms


In [54]:
# 5. get the 100-highest scored candidates as keyphrases
keyphrases_positionrank = extractor_positionrank.get_n_best(n=100)
keyphrases_positionrank

[('электив - чудо', 0.04865810778531971),
 ('электив - пытка', 0.048601395976591265),
 ('электив -', 0.048466308648122486),
 ('данный электив -', 0.048466308648122486),
 ('общем - электив', 0.048466308648122486),
 ('немецкий - электив', 0.048466308648122486),
 ('преподаватель - практик', 0.045315500624895956),
 ('преподаватель - душка', 0.04504210541981257),
 ('преподаватель - психолог', 0.044891110871593046),
 ('преподаватель - итальянец', 0.044854606113516556),
 ('преподаватель - крутая', 0.044852413137503155),
 ('преподаватель - замечательная', 0.04485014542185308),
 ('преподаватель -', 0.04476368427775486),
 ('- замечательный преподаватель', 0.04476368427775486),
 ('последняя - преподаватель', 0.04476368427775486),
 ('сам преподаватель -', 0.04476368427775486),
 ('практики -', 0.042795125884149965),
 ('- весёлые практики', 0.042795125884149965),
 ('решение -', 0.04078299110904044),
 ('баллы -', 0.03737303945652825),
 ('замечательный преподаватель практики', 0.035407562420353575),
 

# MultipartiteRank

In [55]:
%%time

# 1. create a MultipartiteRank extractor.
extractor_multipartiterank = pke.unsupervised.MultipartiteRank()

stoplist = list(string.punctuation)
stoplist += pke.lang.stopwords.get('ru')

# 2. load the content of the document.
extractor_multipartiterank.load_document(input=text,
                                         language="ru",
                                         stoplist=stoplist)

# 3. select the longest sequences of nouns and adjectives, that do
#    not contain punctuation marks or stopwords as candidates.
pos = {'NOUN', 'PROPN'}
extractor_multipartiterank.candidate_selection(pos=pos)

# 4. build the Multipartite graph and rank candidates using random
#    walk, alpha controls the weight adjustment mechanism, see
#    TopicRank for threshold/method parameters.
extractor_multipartiterank.candidate_weighting(alpha=1.5,
                                               threshold=0.25,
                                               method='average')


CPU times: user 3min 28s, sys: 2.94 s, total: 3min 31s
Wall time: 3min 31s


In [56]:
# 5. get the 100-highest scored candidates as keyphrases
keyphrases_multipartiterank = extractor_multipartiterank.get_n_best(n=100)
keyphrases_multipartiterank

[('электив', 0.04366968008791535),
 ('преподаватель', 0.029052902571779293),
 ('пары', 0.026348506959864387),
 ('задания', 0.01882588699619859),
 ('лекции', 0.01401080222374473),
 ('баллы', 0.01267645705351588),
 ('работы', 0.011460256698121481),
 ('тему', 0.011015522535422546),
 ('элективом', 0.010614899230453605),
 ('занятия', 0.010054305019815116),
 ('практики', 0.00932047923995928),
 ('курса', 0.008613943996838519),
 ('студентам', 0.00849804935722269),
 ('информации', 0.008231521788113559),
 ('презентации', 0.008070599446386283),
 ('зачёт', 0.007795361237515503),
 ('тест', 0.007423200436219892),
 ('вопросы', 0.007116353445979112),
 ('материал', 0.006758269606736994),
 ('знаний', 0.006449717215196795),
 ('предмета', 0.005557074458146722),
 ('целом', 0.005529209491101293),
 ('языке', 0.004640493369427495),
 ('группы', 0.004626151577285963),
 ('проект', 0.004623502161208501),
 ('время', 0.004042613657103062),
 ('фильмы', 0.003950389819192411),
 ('преподавательница', 0.0038461290267977